# Notebook 7: Merged Data Exploration

Now I want to start working with merging the data to explore trends over time. This will be the basis of my story. 

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import glob

#import energy usage data of Philadelphia's buildings from 2013-2017
yr2013_df = pd.read_csv('../data/energy_usage_large_commercial_buildings_reported_2013.csv')
yr2014_df = pd.read_csv('../data/energy_usage_large_commercial_buildings_reported_2014.csv')
yr2015_df = pd.read_csv('../data/properties_reported_2015.csv')
yr2016_df = pd.read_csv('../data/properties_reported_2016.csv')
yr2017_df = pd.read_csv('../data/properties_reported_2017.csv')


## 1. Explore headers to find common ID

First, I need to find a common ID to merge the dataframes with


In [2]:
yr2017_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,2310408.2,3589,1100 CATHARINE ST,Palumbo,781043800,19147,1,1980,K-12 School,185206.0,3330829,407768.5,4970622.0,NaN,97.0,41.5,68.6,672.8,646.8,NaN
1,23297409.0,4560,4100 CITY AVE,WPVI-TV,883080200,19131,1,2009,Office,69698.0,3681504,1242178.2,NaN,NaN,1.0,352.1,1068.3,2322.0,2650.4,NaN
2,NaN,4573,4343 WHITAKER AVE,ECI Building,884184500,19124,1,1950,Distribution Center,93000.0,3956329,663312.3,NaN,NaN,100.0,7.1,7.5,68.1,35.2,NaN
3,4749439.2,4153,3801 MARKET ST,First District Plaza,883805600,19104,1,1986,Other,87830.0,6432105,432235.7,NaN,NaN,NaN,59.3,175.9,1838.1,549.8,NaN
4,1168752.3,3450,1100 TYSON AVE,Moore,786543400,19111,1,1980,K-12 School,67701.0,3490894,3762754.7,NaN,NaN,32.0,72.8,112.6,1456.4,329.5,NaN


It seems as if every building as an object id

In [3]:
yr2016_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,22205732.7,5260,233-43 S BROAD ST,Doubletree Hotel,883702001,19107,1,1983,Hotel,565764.0,3774262,10622801.8,NaN,NaN,78.0,58.0,143.0,23640.1,3110.8,NaN
1,1160875.0,6195,700-50 CATHEDRAL RD,Cathedral East Apartments,881088500,19128,1,1964,Multifamily Housing,92156.0,5140660,4181930.1,NaN,NaN,84.0,58.0,87.2,2190.6,355.3,NaN
2,680100.7,6119,8020 ALGON AVE,8020 Algon,881183500,19152,1,1964,Multifamily Housing,50840.0,5029250,1824667.9,NaN,NaN,95.0,49.3,79.7,1021.8,174.9,NaN
3,1564263.5,4993,926-76 E CHELTEN AVE,Pastorius,785099400,19138,1,1967,K-12 School,75318.0,3337464,2648097.0,NaN,NaN,86.0,55.9,102.1,3146.3,320.0,NaN
4,584826.7,5327,6951 NEW STATE RD,MARJAM SUPPLY,884182000,19135,1,1945,Distribution Center,106400.0,3813146,467121.4,NaN,NaN,91.0,9.9,21.9,612.2,91.9,NaN


In [4]:
yr2015_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,4192843.0,2806,4501 BENNER ST,Lawton,786033600,19135,1,1973,K-12 School,79856.0,3330959,173627.5,NaN,NaN,37.0,54.7,167.1,870.9,570.5,NaN
1,1061172.9,3551,500-50 E ERIE AVE,500 Erie Ave,884048000,19134,1,1930,Distribution Center,130000.0,4104348,NaN,NaN,NaN,86.0,8.2,25.6,154.4,142.1,NaN
2,35489338.0,2841,3701 MARKET ST,3701 Market St.,883073300,19104,1,2000,Medical Office,140913.0,3340008,NaN,NaN,NaN,1.0,251.9,790.8,6528.2,4750.8,NaN
3,4934315.0,3229,1600-46 RACE ST,Friends Select School,772199000,19103,1,1969,K-12 School,123306.0,3891486,6447756.2,NaN,NaN,39.0,92.3,180.6,1083.3,1003.0,NaN
4,6759973.8,3103,14700 TOWNSEND RD,NARA,884292516,19154,1,1994,Other - Public Services,300000.0,3790331,4453888.7,NaN,NaN,NaN,37.4,86.3,103.9,1141.5,NaN


In [5]:
yr2014_df.head()

,natural_gas_use_kbtu,portfolio_manager_id,postal_code,property_name,source_eui_kbtu_ft,total_ghg_emissions_mtco2e,district_steam_use_kbtu,site_eui_kbtu_ft,primary_property_type_epa_calculated,number_of_buildings,energy_star_score,location_1_address,year_built,fuel_oil_2_use_kbtu,property_floor_area_buildings_and_parking_ft,water_use_all_water_sources_kgal,electricity_use_kbtu,notes,philadelphia_building_id
0,"5,108,484.20",3330960,19120.0,Lowell,245.2,651.0,Not Available,136.5,K-12 School,1,11,450 W NEDRO AVE,1914,Not Available,58200.0,1493.9,2836085.4,NaN,785548000.0
1,"6,566,999.70",3952012,19116.0,10814 NE AVENUE,35.6,715.1,Not Available,21.4,Other - Services,1,Not Available,10814 NE AVENUE,1979,Not Available,217560.0,245.5,2736515.4,NaN,884240500.0
2,"243,912.00",3989966,19124.0,Juniata,34.1,88.9,Not Available,13.6,Self-Storage Facility,1,Not Available,"4123 ""G"" St.",2005,Not Available,59850.0,19.7,567623.4,NaN,884114205.0
3,"5,401,817.60",3330939,19143.0,Dobson,146.5,373.0,Not Available,115.1,K-12 School,1,12,4667 UMBRIA,1930,Not Available,52500.0,379.5,643237.4,NaN,784463600.0
4,"78,090.40",3330927,19128.0,Levering,121.5,546.1,Not Available,97.6,K-12 School,1,54,6000 RIDGE AVE,1929,"5,955,528.10",69475.0,284.1,747159.8,NaN,784378000.0


In [6]:
yr2013_df.head()

,the_geom,lng,year_built,electricity_use_grid_purchase_and_generated_from_onsite_renewab,energy_star_score,portfolio_manager_id,water_use_all_water_sources_kgal,district_steam_use_kbtu,site_eui_kbtu_ft2,philadelphia_building_id,...,fuel_oil_2_use_kbtu,number_of_buildings,property_floor_area_buildings_and_parking_ft,postal_code,natural_gas_use_kbtu,notes,total_ghg_emissions_mtco2e,primary_property_type_epa_calculated,lat,source_eui_kbtu_ft2
0,0101000020E61000002CBA8FB8D4CA52C0C82FAB660CFE...,-75.169233,1940,2146869.3,25,3330868,1604.5,Not Available,94.0,784321800.0,...,Not Available,1,109651.0,19121.0,8157192.2,NaN,704.8,K-12 School,39.984753,139.6
1,0101000020E610000080FC0401C6CB52C0A0BF7A90E9F9...,-75.183960,1917,3916010.4,73,3443035,2900.9,1299111.2,67.8,773717200.0,...,Not Available,1,79200.0,19104.0,155304.3,NaN,619.1,Office,39.952440,177.0
2,0101000020E6100000DC15D69AE1CA52C068520C2805F9...,-75.170020,1905,13312010.1,1,3512105,262.0,Not Available,225.2,772614900.0,...,0,1,59101.0,19146.0,Not Available,NaN,1685.8,Office,39.945470,707.3
3,0101000020E6100000B09CFBE0B9BF52C0F07C5E429C0E...,-74.995720,1950,192780.4,100,3892836,28.4,Not Available,3.0,884233835.0,...,Not Available,1,64638.0,19116.0,0,NaN,24.4,Office,40.114144,9.4
4,0101000020E610000054D00AC543CA52C09851DFDEA7F9...,-75.160386,1925,2756896.0,44,3759390,985.2,558808.5,69.5,883311000.0,...,Not Available,1,51676.0,19107.0,274307.8,NaN,413.2,Office,39.950436,186.1


Here I see that each building has an account number / property building id. This will be the common thread I use to merge my dataframes into my main source.

## 2. Clean up the id column 
I want to clean up the id column. Here you can see that the ID's in 2013 and 2014 are called "Philadelphia Building ID" and have a decimals place. I will want 2013- 2017 to all be labeled "building ID" without decimals. 

### 2017 Clean Up

In [7]:
yr2017_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,2310408.2,3589,1100 CATHARINE ST,Palumbo,781043800,19147,1,1980,K-12 School,185206.0,3330829,407768.5,4970622.0,NaN,97.0,41.5,68.6,672.8,646.8,NaN
1,23297409.0,4560,4100 CITY AVE,WPVI-TV,883080200,19131,1,2009,Office,69698.0,3681504,1242178.2,NaN,NaN,1.0,352.1,1068.3,2322.0,2650.4,NaN
2,NaN,4573,4343 WHITAKER AVE,ECI Building,884184500,19124,1,1950,Distribution Center,93000.0,3956329,663312.3,NaN,NaN,100.0,7.1,7.5,68.1,35.2,NaN
3,4749439.2,4153,3801 MARKET ST,First District Plaza,883805600,19104,1,1986,Other,87830.0,6432105,432235.7,NaN,NaN,NaN,59.3,175.9,1838.1,549.8,NaN
4,1168752.3,3450,1100 TYSON AVE,Moore,786543400,19111,1,1980,K-12 School,67701.0,3490894,3762754.7,NaN,NaN,32.0,72.8,112.6,1456.4,329.5,NaN


Here we want to focus on cleaning up the opa account num

First, lets get rid of the NA's

In [79]:
yr2017_df=yr2017_df[(-yr2017_df['opa_account_num'].isnull()) & (yr2017_df['opa_account_num']!='Many')]

Now lets make sure these are all integers

In [9]:
yr2017_df[yr2017_df['opa_account_num']=='777509000']

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
1057,3021868.8,3306,1801-35 N BROAD ST,CONWELL HALL,777509000,19122,1,2000,College/University,67644.0,3979396,261.8,NaN,5197131.1,NaN,161.9,273.1,13419.9,824.2,NaN
1060,5800324.9,3809,1801-35 N BROAD ST,SPEAKMAN,777509000,19122,1,2000,College/University,110605.0,3979395,70.3,NaN,3225808.1,NaN,145.9,264.1,456.4,1232.4,NaN
1073,9770923.9,4241,1819 N BROAD ST,WACHMAN HALL,777509000,19122,1,2000,College/University,130655.0,3983810,6749.3,NaN,8011081.3,NaN,194.0,366.5,3193.1,2014.8,NaN
1109,2310704.7,3590,1810 LIACOURAS WALK,1810 LIACOURAS WALK,777509000,19122,1,2000,College/University,74630.0,3983897,NaN,NaN,24544828.2,NaN,367.6,500.6,1190.3,1916.6,NaN
1112,3392066.8,3771,1801-35 N BROAD ST,CARNELL HALL,777509000,19122,1,2000,College/University,75931.0,3979397,NaN,NaN,4209391.7,NaN,144.3,251.1,3994.2,832.5,NaN
1128,7848575.8,4435,1801 LIACOURAS WALK,Alter Hall,777509000,19122,1,2000,College/University,217526.0,3983809,NaN,NaN,9287213.2,NaN,127.2,213.1,2801.8,2043.0,NaN


Here we see that there are portfolios of buildings with the same ID. For the sake of my analysis, I want to exclude them from my final dataframe. 

In [10]:
building_id_2017=yr2017_df['opa_account_num'].value_counts().reset_index()

In [11]:
building_id_2017.head()

,index,opa_account_num
0,777509000,6
1,771020000,4
2,777568800,4
3,786504751,4
4,778503000,4


In [12]:
buildings_to_use_2017=building_id_2017[building_id_2017['opa_account_num']==1]['index']

In [13]:
yr2017_subset_df=yr2017_df[yr2017_df['opa_account_num'].isin(buildings_to_use_2017)]

In [14]:
yr2017_subset_df['opa_account_num'].value_counts()

773080515                                                1
888140126                                                1
883703900                                                1
884671900                                                1
882125300                                                1
888461228                                                1
881551130                                                1
774688100                                                1
785356200                                                1
777575905                                                1
883338000                                                1
882059510                                                1
884202100                                                1
883430000                                                1
884233850                                                1
783130400                                                1
884243500                                               

Here we have a final dataframe of 1,454 buildings -- all in integer form

### 2016 Clean Up

In [15]:
yr2016_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,22205732.7,5260,233-43 S BROAD ST,Doubletree Hotel,883702001,19107,1,1983,Hotel,565764.0,3774262,10622801.8,NaN,NaN,78.0,58.0,143.0,23640.1,3110.8,NaN
1,1160875.0,6195,700-50 CATHEDRAL RD,Cathedral East Apartments,881088500,19128,1,1964,Multifamily Housing,92156.0,5140660,4181930.1,NaN,NaN,84.0,58.0,87.2,2190.6,355.3,NaN
2,680100.7,6119,8020 ALGON AVE,8020 Algon,881183500,19152,1,1964,Multifamily Housing,50840.0,5029250,1824667.9,NaN,NaN,95.0,49.3,79.7,1021.8,174.9,NaN
3,1564263.5,4993,926-76 E CHELTEN AVE,Pastorius,785099400,19138,1,1967,K-12 School,75318.0,3337464,2648097.0,NaN,NaN,86.0,55.9,102.1,3146.3,320.0,NaN
4,584826.7,5327,6951 NEW STATE RD,MARJAM SUPPLY,884182000,19135,1,1945,Distribution Center,106400.0,3813146,467121.4,NaN,NaN,91.0,9.9,21.9,612.2,91.9,NaN


In [16]:
yr2015_df=yr2016_df[(-yr2016_df['opa_account_num'].isnull()) & (yr2016_df['opa_account_num']!='Many')]

In [17]:
yr2016_df[yr2016_df['opa_account_num']=='777509000']

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
1126,4390046.4,5610,1801-35 N BROAD ST,SPEAKMAN,777509000,19122,1,2000,College/University,110605.0,3979395,146.7,NaN,1260099.8,NaN,94.2,181.4,30.3,838.4,NaN
1137,3034291.6,5547,1801-35 N BROAD ST,CONWELL HALL,777509000,19122,1,2000,College/University,67644.0,3979396,300.9,NaN,6497449.4,NaN,184.0,299.5,12122.5,933.1,NaN
1138,8411395.5,5553,1819 N BROAD ST,WACHMAN HALL,777509000,19122,1,2000,College/University,130655.0,3983810,101.7,NaN,8332269.0,NaN,171.3,322.0,11061.1,1814.7,NaN
1162,3406025.6,5548,1801-35 N BROAD ST,CARNELL HALL,777509000,19122,1,2000,College/University,75931.0,3979397,NaN,NaN,3825074.8,NaN,138.3,244.6,418.8,817.1,NaN
1163,8391752.6,5552,1801 LIACOURAS WALK,Alter Hall,777509000,19122,1,2000,College/University,217526.0,3983809,NaN,NaN,12098181.2,NaN,165.2,259.1,3200.0,2579.8,NaN
1164,2687022.3,5558,1810 LIACOURAS WALK,1810 LIACOURAS WALK,777509000,19122,1,2000,College/University,74630.0,3983897,NaN,NaN,13597752.6,NaN,229.0,343.0,1167.5,1253.3,NaN


In [18]:
building_id_2016=yr2016_df['opa_account_num'].value_counts().reset_index()

In [19]:
building_id_2016.head()

,index,opa_account_num
0,777509000,6
1,786504751,4
2,771020000,4
3,778503000,4
4,777568800,4


In [20]:
buildings_to_use_2016=building_id_2016[building_id_2016['opa_account_num']==1]['index']

In [21]:
yr2016_subset_df=yr2016_df[yr2016_df['opa_account_num'].isin(buildings_to_use_2016)]

In [22]:
yr2016_subset_df['opa_account_num'].value_counts()

884843500                                                1
882059510                                                1
884671900                                                1
881025000                                                1
882125300                                                1
786082700                                                1
881000151                                                1
774688100                                                1
785356200                                                1
777575905                                                1
883338000                                                1
886668100                                                1
883032500                                                1
884167024                                                1
884233850                                                1
783130400                                                1
784472600                                               

2016 has a final list of 1,483 buildings after this clean up

### 2015 Clean Up

In [23]:
yr2015_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,22205732.7,5260,233-43 S BROAD ST,Doubletree Hotel,883702001,19107,1,1983,Hotel,565764.0,3774262,10622801.8,NaN,NaN,78.0,58.0,143.0,23640.1,3110.8,NaN
1,1160875.0,6195,700-50 CATHEDRAL RD,Cathedral East Apartments,881088500,19128,1,1964,Multifamily Housing,92156.0,5140660,4181930.1,NaN,NaN,84.0,58.0,87.2,2190.6,355.3,NaN
2,680100.7,6119,8020 ALGON AVE,8020 Algon,881183500,19152,1,1964,Multifamily Housing,50840.0,5029250,1824667.9,NaN,NaN,95.0,49.3,79.7,1021.8,174.9,NaN
3,1564263.5,4993,926-76 E CHELTEN AVE,Pastorius,785099400,19138,1,1967,K-12 School,75318.0,3337464,2648097.0,NaN,NaN,86.0,55.9,102.1,3146.3,320.0,NaN
4,584826.7,5327,6951 NEW STATE RD,MARJAM SUPPLY,884182000,19135,1,1945,Distribution Center,106400.0,3813146,467121.4,NaN,NaN,91.0,9.9,21.9,612.2,91.9,NaN


In [24]:
yr2015_df['opa_account_num'].value_counts()

777509000                                                6
777568800                                                4
778503000                                                4
771020000                                                4
786504751                                                4
881299200                                                3
777570600                                                3
881008255                                                3
778144000                                                3
777570200                                                2
774502000                                                2
882040300                                                2
884201700                                                2
777573005                                                2
Multiple                                                 2
775165500                                                2
786264650                                               

There are clearly some buildings with same account number. Let's explore further to see what is happening. Also, we see something that says "many" which we want to remove.

In [25]:
yr2015_df=yr2015_df[(-yr2015_df['opa_account_num'].isnull()) & (yr2015_df['opa_account_num']!='Many')]

In [26]:
yr2015_df[yr2015_df['opa_account_num']=='777509000']

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
1126,4390046.4,5610,1801-35 N BROAD ST,SPEAKMAN,777509000,19122,1,2000,College/University,110605.0,3979395,146.7,NaN,1260099.8,NaN,94.2,181.4,30.3,838.4,NaN
1137,3034291.6,5547,1801-35 N BROAD ST,CONWELL HALL,777509000,19122,1,2000,College/University,67644.0,3979396,300.9,NaN,6497449.4,NaN,184.0,299.5,12122.5,933.1,NaN
1138,8411395.5,5553,1819 N BROAD ST,WACHMAN HALL,777509000,19122,1,2000,College/University,130655.0,3983810,101.7,NaN,8332269.0,NaN,171.3,322.0,11061.1,1814.7,NaN
1162,3406025.6,5548,1801-35 N BROAD ST,CARNELL HALL,777509000,19122,1,2000,College/University,75931.0,3979397,NaN,NaN,3825074.8,NaN,138.3,244.6,418.8,817.1,NaN
1163,8391752.6,5552,1801 LIACOURAS WALK,Alter Hall,777509000,19122,1,2000,College/University,217526.0,3983809,NaN,NaN,12098181.2,NaN,165.2,259.1,3200.0,2579.8,NaN
1164,2687022.3,5558,1810 LIACOURAS WALK,1810 LIACOURAS WALK,777509000,19122,1,2000,College/University,74630.0,3983897,NaN,NaN,13597752.6,NaN,229.0,343.0,1167.5,1253.3,NaN


Here we can see that there are some buildings with the same account number. For example, OPA account number #777509000 led to 6 different buildings all in the same geographic area on Temple university. We are assuming that this is a portfolio of buildings. For the sake of my analysis, I want to focus my data on just those with a unique id number.

In [27]:
building_id_2015=yr2015_df['opa_account_num'].value_counts().reset_index()

In [28]:
building_id_2015.head()

,index,opa_account_num
0,777509000,6
1,777568800,4
2,778503000,4
3,771020000,4
4,786504751,4


In [29]:
buildings_to_use_2015=building_id_2015[building_id_2015['opa_account_num']==1]['index']

In [30]:
yr2015_subset_df=yr2015_df[yr2015_df['opa_account_num'].isin(buildings_to_use_2015)]

In [31]:
yr2015_subset_df['opa_account_num'].value_counts()

884843500                                                1
882059510                                                1
884671900                                                1
881025000                                                1
882125300                                                1
786082700                                                1
881000151                                                1
774688100                                                1
785356200                                                1
777575905                                                1
883338000                                                1
886668100                                                1
883032500                                                1
884167024                                                1
884233850                                                1
783130400                                                1
784472600                                               

Now I have my new data-frame with unique ID numbers. The 2015 dataframe has 1064 buildings

### 2014 Cleanup

In [32]:
yr2014_df.head()

,natural_gas_use_kbtu,portfolio_manager_id,postal_code,property_name,source_eui_kbtu_ft,total_ghg_emissions_mtco2e,district_steam_use_kbtu,site_eui_kbtu_ft,primary_property_type_epa_calculated,number_of_buildings,energy_star_score,location_1_address,year_built,fuel_oil_2_use_kbtu,property_floor_area_buildings_and_parking_ft,water_use_all_water_sources_kgal,electricity_use_kbtu,notes,philadelphia_building_id
0,"5,108,484.20",3330960,19120.0,Lowell,245.2,651.0,Not Available,136.5,K-12 School,1,11,450 W NEDRO AVE,1914,Not Available,58200.0,1493.9,2836085.4,NaN,785548000.0
1,"6,566,999.70",3952012,19116.0,10814 NE AVENUE,35.6,715.1,Not Available,21.4,Other - Services,1,Not Available,10814 NE AVENUE,1979,Not Available,217560.0,245.5,2736515.4,NaN,884240500.0
2,"243,912.00",3989966,19124.0,Juniata,34.1,88.9,Not Available,13.6,Self-Storage Facility,1,Not Available,"4123 ""G"" St.",2005,Not Available,59850.0,19.7,567623.4,NaN,884114205.0
3,"5,401,817.60",3330939,19143.0,Dobson,146.5,373.0,Not Available,115.1,K-12 School,1,12,4667 UMBRIA,1930,Not Available,52500.0,379.5,643237.4,NaN,784463600.0
4,"78,090.40",3330927,19128.0,Levering,121.5,546.1,Not Available,97.6,K-12 School,1,54,6000 RIDGE AVE,1929,"5,955,528.10",69475.0,284.1,747159.8,NaN,784378000.0


Here we see that we need to focus on the Philadlphia Building ID

In [33]:
yr2014_df['philadelphia_building_id'].value_counts()

777509000.0    6
786504751.0    4
771020000.0    4
777568800.0    3
777570400.0    2
777569000.0    2
771019000.0    2
771013000.0    2
786264650.0    2
776516000.0    2
785056600.0    2
882125300.0    2
777570200.0    2
777573005.0    2
773502090.0    2
778503000.0    2
883102000.0    2
774569000.0    1
881449900.0    1
881449800.0    1
883705100.0    1
773010000.0    1
881447300.0    1
881445900.0    1
881444600.0    1
884269100.0    1
881439700.0    1
883056905.0    1
772534700.0    1
881436700.0    1
              ..
884213400.0    1
882117000.0    1
883707000.0    1
775165500.0    1
884245510.0    1
884244700.0    1
785689600.0    1
781092450.0    1
884242000.0    1
884240500.0    1
884239600.0    1
884288400.0    1
884236410.0    1
884236000.0    1
884233850.0    1
884232400.0    1
783568200.0    1
773080515.0    1
882131101.0    1
884228000.0    1
884291400.0    1
771251200.0    1
884130500.0    1
785657200.0    1
882125404.0    1
884221900.0    1
884220500.0    1
884219050.0   

Here we see multiple buildings again. Also, we see that we want to change the dataframe into integers.

In [34]:
yr2014_df=yr2014_df[-yr2014_df['philadelphia_building_id'].isnull()]

In [35]:
yr2014_df['philadelphia_building_id']=yr2014_df['philadelphia_building_id'].astype('int')

In [36]:
yr2014_df[yr2014_df['philadelphia_building_id']==777509000]

,natural_gas_use_kbtu,portfolio_manager_id,postal_code,property_name,source_eui_kbtu_ft,total_ghg_emissions_mtco2e,district_steam_use_kbtu,site_eui_kbtu_ft,primary_property_type_epa_calculated,number_of_buildings,energy_star_score,location_1_address,year_built,fuel_oil_2_use_kbtu,property_floor_area_buildings_and_parking_ft,water_use_all_water_sources_kgal,electricity_use_kbtu,notes,philadelphia_building_id
37,"6,463.20",3979396,19122.0,CONWELL HALL,342.4,1133.1,"6,528,016.20",215.6,College/University,1,Not Available,NE CORNER BROAD AND MONTGOMERY,2000,Not Available,67644.0,11683.4,3389706.0,NaN,777509000
38,Not Available,3979397,19122.0,CARNELL HALL,301.0,1098.3,"4,718,652.30",181.1,College/University,1,Not Available,NE CORNER BROAD AND MONTGOMERY,2000,Not Available,75931.0,1018.9,3804977.1,NaN,777509000
349,Not Available,3983809,19122.0,Alter Hall,200.5,2095.2,"9,738,168.30",119.0,College/University,1,Not Available,1801 Liacouras Walk,2000,Not Available,217526.0,2982.4,7365624.3,NaN,777509000
372,"3,270.30",3983810,19122.0,WACHMAN HALL,339.5,2107.5,"8,322,311.30",194.4,College/University,1,Not Available,1819 N. BROAD ST.,2000,Not Available,130655.0,8430.6,8070495.7,NaN,777509000
504,Not Available,3983897,19122.0,1810 LIACOURAS WALK,271.1,999.4,"7,464,589.60",169.8,College/University,1,Not Available,1810 Liacouras Walk,2000,Not Available,74630.0,1085.5,2824825.5,NaN,777509000
637,"3,231.60",3979395,19122.0,SPEAKMAN,265.2,1480.8,"10,151,924.90",188.1,College/University,1,Not Available,1801 N. 13TH,2000,Not Available,110605.0,15.0,3025710.4,NaN,777509000


Here we see the same thing with multiple ID's and thus want to subset these out of my data

In [37]:
building_id_2014=yr2014_df['philadelphia_building_id'].value_counts().reset_index()

In [38]:
building_id_2014.head()

,index,philadelphia_building_id
0,777509000,6
1,786504751,4
2,771020000,4
3,777568800,3
4,777569000,2


In [39]:
buildings_to_use_2014=building_id_2014[building_id_2014['philadelphia_building_id']==1]['index']

In [40]:
yr2014_subset_df=yr2014_df[yr2014_df['philadelphia_building_id'].isin(buildings_to_use_2014)]

Now I want to check that this worked

In [41]:
yr2014_subset_df['philadelphia_building_id'].value_counts()

884206500    1
883710000    1
785531600    1
884822730    1
783782600    1
883524700    1
884839110    1
775329000    1
884130500    1
884177600    1
778038005    1
882125501    1
786503420    1
785097400    1
771251500    1
772025000    1
883704500    1
784167600    1
781609400    1
884220000    1
783183400    1
884259500    1
774024000    1
883026600    1
883073700    1
771005500    1
784048800    1
771015010    1
786146001    1
883071700    1
            ..
774005000    1
881436010    1
783707500    1
783431200    1
784472600    1
884019029    1
883703600    1
772535700    1
884099200    1
783181200    1
786249100    1
881147275    1
884346250    1
884186505    1
882245000    1
776251000    1
772724100    1
788013000    1
884161920    1
883705215    1
785202400    1
778501500    1
883400500    1
771739000    1
788008310    1
882070900    1
786462480    1
772147500    1
785657200    1
0            1
Name: philadelphia_building_id, Length: 946, dtype: int64

The 2014 subsetted dataframe is now 946 buildings and in integer form

### 2013 cleanup

In [42]:
yr2013_df.head()

,the_geom,lng,year_built,electricity_use_grid_purchase_and_generated_from_onsite_renewab,energy_star_score,portfolio_manager_id,water_use_all_water_sources_kgal,district_steam_use_kbtu,site_eui_kbtu_ft2,philadelphia_building_id,...,fuel_oil_2_use_kbtu,number_of_buildings,property_floor_area_buildings_and_parking_ft,postal_code,natural_gas_use_kbtu,notes,total_ghg_emissions_mtco2e,primary_property_type_epa_calculated,lat,source_eui_kbtu_ft2
0,0101000020E61000002CBA8FB8D4CA52C0C82FAB660CFE...,-75.169233,1940,2146869.3,25,3330868,1604.5,Not Available,94.0,784321800.0,...,Not Available,1,109651.0,19121.0,8157192.2,NaN,704.8,K-12 School,39.984753,139.6
1,0101000020E610000080FC0401C6CB52C0A0BF7A90E9F9...,-75.183960,1917,3916010.4,73,3443035,2900.9,1299111.2,67.8,773717200.0,...,Not Available,1,79200.0,19104.0,155304.3,NaN,619.1,Office,39.952440,177.0
2,0101000020E6100000DC15D69AE1CA52C068520C2805F9...,-75.170020,1905,13312010.1,1,3512105,262.0,Not Available,225.2,772614900.0,...,0,1,59101.0,19146.0,Not Available,NaN,1685.8,Office,39.945470,707.3
3,0101000020E6100000B09CFBE0B9BF52C0F07C5E429C0E...,-74.995720,1950,192780.4,100,3892836,28.4,Not Available,3.0,884233835.0,...,Not Available,1,64638.0,19116.0,0,NaN,24.4,Office,40.114144,9.4
4,0101000020E610000054D00AC543CA52C09851DFDEA7F9...,-75.160386,1925,2756896.0,44,3759390,985.2,558808.5,69.5,883311000.0,...,Not Available,1,51676.0,19107.0,274307.8,NaN,413.2,Office,39.950436,186.1


Once again, like 2014 we want to use the Philadelphia Building ID and convert into an integer.

In [43]:
yr2013_df=yr2013_df[(-yr2013_df['philadelphia_building_id'].isnull()) ]

Now we need to put it into integer form to get rid of the .00

In [44]:
yr2013_df['philadelphia_building_id']=yr2013_df['philadelphia_building_id'].astype('int')

In [45]:
building_id_2013=yr2013_df['philadelphia_building_id'].value_counts().reset_index()

In [46]:
building_id_2013.head()

,index,philadelphia_building_id
0,777509000,6
1,778144000,3
2,771020000,3
3,778503000,3
4,786504751,3


In [47]:
buildings_to_use_2013=building_id_2013[building_id_2013['philadelphia_building_id']==1]['index']

In [48]:
yr2013_subset_df=yr2013_df[yr2013_df['philadelphia_building_id'].isin(buildings_to_use_2013)]

In [49]:
yr2013_subset_df['philadelphia_building_id'].value_counts()

884117500    1
884351660    1
883405500    1
772018000    1
882078060    1
785097400    1
884119600    1
882938550    1
883704500    1
783576000    1
884161200    1
781609400    1
884202501    1
884284000    1
788023020    1
883409600    1
884153000    1
884247200    1
884220900    1
884242000    1
784552600    1
882842425    1
777530000    1
884461920    1
883407500    1
774517200    1
884069000    1
883053530    1
883026600    1
773743300    1
            ..
783568200    1
882120009    1
784168800    1
883365200    1
883056000    1
778651005    1
778501500    1
772513020    1
788020600    1
788008310    1
882070900    1
786462480    1
884163400    1
785657200    1
783707500    1
884346250    1
881436010    1
774005000    1
774385000    1
883080550    1
774688100    1
775165500    1
884080300    1
884094300    1
772607000    1
783072600    1
884188500    1
778653010    1
772541000    1
0            1
Name: philadelphia_building_id, Length: 949, dtype: int64

Now we have our final dataframe for 2013. This one has 949 buildings.

## 3. Rename columns so that they are all the same

In [50]:
print(yr2017_subset_df.columns)

Index(['electricity_use_kbtu', 'objectid', 'street_address', 'property_name',
       'opa_account_num', 'postal_code', 'num_of_buildings', 'year_built',
       'primary_prop_type_epa_calc', 'total_floor_area_bld_pk_ft2',
       'portfolio_manager_id', 'natural_gas_use_kbtu', 'fuel_oil_o2_use_kbtu',
       'steam_use_kbtu', 'energy_star_score', 'site_eui_kbtuft2',
       'source_eui_kbtuft2', 'water_use_all_kgal',
       'total_ghg_emissions_mtco2e', 'notes'],
      dtype='object')


In [63]:
yr2017_subset_df.rename(columns={'electricity_use_kbtu': 'Electricity Use', 'objectid': 'Object ID', 'street_address': 'Street Address', 'property_name': 'Property Name', 'opa_account_num': 'Building ID', 'postal_code': 'Postal Code', 'num_of_buildings': 'Num of Buildings', 'year_built': 'Year Built', 'primary_prop_type_epa_calc': 'Property Type', 'total_floor_area_bld_pk_ft2': 'Square Footage', 'portfolio_manager_id': 'Port Manager ID', 'natural_gas_use_kbtu': 'Natural Gas Use', 'fuel_oil_o2_use_kbtu': 'Fuel Oil Use', 'steam_use_kbtu': 'Steam Use', 'energy_star_score': 'Energy Star Score', 'site_eui_kbtuft2': 'Site EUI', 'source_eui_kbtuft2': 'Source EUI', 'water_use_all_kgal': 'Water Use', 'total_ghg_emissions_mtco2e': 'Total Greenhouse Gas Emission', 'notes': 'notes'}, inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [64]:
yr2017_subset_df.head()

,Electricity Use,Object ID,Street Address,Property Name,Building ID,Postal Code,Num of Buildings,Year Built,Property Type,Square Footage,Port Manager ID,Natural Gas Use,Fuel Oil Use,Steam Use,Energy Star Score,Site EUI,Source EUI,Water Use,Total Greenhouse Gas Emission,notes
0,2310408.2,3589,1100 CATHARINE ST,Palumbo,781043800,19147,1,1980,K-12 School,185206.0,3330829,407768.5,4970622.0,NaN,97.0,41.5,68.6,672.8,646.8,NaN
1,23297409.0,4560,4100 CITY AVE,WPVI-TV,883080200,19131,1,2009,Office,69698.0,3681504,1242178.2,NaN,NaN,1.0,352.1,1068.3,2322.0,2650.4,NaN
2,NaN,4573,4343 WHITAKER AVE,ECI Building,884184500,19124,1,1950,Distribution Center,93000.0,3956329,663312.3,NaN,NaN,100.0,7.1,7.5,68.1,35.2,NaN
3,4749439.2,4153,3801 MARKET ST,First District Plaza,883805600,19104,1,1986,Other,87830.0,6432105,432235.7,NaN,NaN,NaN,59.3,175.9,1838.1,549.8,NaN
4,1168752.3,3450,1100 TYSON AVE,Moore,786543400,19111,1,1980,K-12 School,67701.0,3490894,3762754.7,NaN,NaN,32.0,72.8,112.6,1456.4,329.5,NaN


In [52]:
print(yr2016_subset_df.columns)

Index(['electricity_use_kbtu', 'objectid', 'street_address', 'property_name',
       'opa_account_num', 'postal_code', 'num_of_buildings', 'year_built',
       'primary_prop_type_epa_calc', 'total_floor_area_bld_pk_ft2',
       'portfolio_manager_id', 'natural_gas_use_kbtu', 'fuel_oil_o2_use_kbtu',
       'steam_use_kbtu', 'energy_star_score', 'site_eui_kbtuft2',
       'source_eui_kbtuft2', 'water_use_all_kgal',
       'total_ghg_emissions_mtco2e', 'notes'],
      dtype='object')


In [65]:
yr2016_subset_df.rename(columns={'electricity_use_kbtu': 'Electricity Use', 'objectid': 'Object ID', 'street_address': 'Street Address', 'property_name': 'Property Name', 'opa_account_num': 'Building ID', 'postal_code': 'Postal Code', 'num_of_buildings': 'Num of Buildings', 'year_built': 'Year Built', 'primary_prop_type_epa_calc': 'Property Type', 'total_floor_area_bld_pk_ft2': 'Square Footage', 'portfolio_manager_id': 'Port Manager ID', 'natural_gas_use_kbtu': 'Natural Gas Use', 'fuel_oil_o2_use_kbtu': 'Fuel Oil Use', 'steam_use_kbtu': 'Steam Use', 'energy_star_score': 'Energy Star Score', 'site_eui_kbtuft2': 'Site EUI', 'source_eui_kbtuft2': 'Source EUI', 'water_use_all_kgal': 'Water Use', 'total_ghg_emissions_mtco2e': 'Total Greenhouse Gas Emission', 'notes': 'notes'}, inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [66]:
yr2016_subset_df.head()

,Electricity Use,Object ID,Street Address,Property Name,Building ID,Postal Code,Num of Buildings,Year Built,Property Type,Square Footage,Port Manager ID,Natural Gas Use,Fuel Oil Use,Steam Use,Energy Star Score,Site EUI,Source EUI,Water Use,Total Greenhouse Gas Emission,notes
0,22205732.7,5260,233-43 S BROAD ST,Doubletree Hotel,883702001,19107,1,1983,Hotel,565764.0,3774262,10622801.8,NaN,NaN,78.0,58.0,143.0,23640.1,3110.8,NaN
1,1160875.0,6195,700-50 CATHEDRAL RD,Cathedral East Apartments,881088500,19128,1,1964,Multifamily Housing,92156.0,5140660,4181930.1,NaN,NaN,84.0,58.0,87.2,2190.6,355.3,NaN
2,680100.7,6119,8020 ALGON AVE,8020 Algon,881183500,19152,1,1964,Multifamily Housing,50840.0,5029250,1824667.9,NaN,NaN,95.0,49.3,79.7,1021.8,174.9,NaN
3,1564263.5,4993,926-76 E CHELTEN AVE,Pastorius,785099400,19138,1,1967,K-12 School,75318.0,3337464,2648097.0,NaN,NaN,86.0,55.9,102.1,3146.3,320.0,NaN
4,584826.7,5327,6951 NEW STATE RD,MARJAM SUPPLY,884182000,19135,1,1945,Distribution Center,106400.0,3813146,467121.4,NaN,NaN,91.0,9.9,21.9,612.2,91.9,NaN


In [53]:
print(yr2015_subset_df.columns)

Index(['electricity_use_kbtu', 'objectid', 'street_address', 'property_name',
       'opa_account_num', 'postal_code', 'num_of_buildings', 'year_built',
       'primary_prop_type_epa_calc', 'total_floor_area_bld_pk_ft2',
       'portfolio_manager_id', 'natural_gas_use_kbtu', 'fuel_oil_o2_use_kbtu',
       'steam_use_kbtu', 'energy_star_score', 'site_eui_kbtuft2',
       'source_eui_kbtuft2', 'water_use_all_kgal',
       'total_ghg_emissions_mtco2e', 'notes'],
      dtype='object')


In [67]:
yr2015_subset_df.rename(columns={'electricity_use_kbtu': 'Electricity Use', 'objectid': 'Object ID', 'street_address': 'Street Address', 'property_name': 'Property Name', 'opa_account_num': 'Building ID', 'postal_code': 'Postal Code', 'num_of_buildings': 'Num of Buildings', 'year_built': 'Year Built', 'primary_prop_type_epa_calc': 'Property Type', 'total_floor_area_bld_pk_ft2': 'Square Footage', 'portfolio_manager_id': 'Port Manager ID', 'natural_gas_use_kbtu': 'Natural Gas Use', 'fuel_oil_o2_use_kbtu': 'Fuel Oil Use', 'steam_use_kbtu': 'Steam Use', 'energy_star_score': 'Energy Star Score', 'site_eui_kbtuft2': 'Site EUI', 'source_eui_kbtuft2': 'Source EUI', 'water_use_all_kgal': 'Water Use', 'total_ghg_emissions_mtco2e': 'Total Greenhouse Gas Emission', 'notes': 'notes'}, inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [68]:
yr2015_subset_df.head()

,Electricity Use,Object ID,Street Address,Property Name,Building ID,Postal Code,Num of Buildings,Year Built,Property Type,Square Footage,Port Manager ID,Natural Gas Use,Fuel Oil Use,Steam Use,Energy Star Score,Site EUI,Source EUI,Water Use,Total Greenhouse Gas Emission,notes
0,22205732.7,5260,233-43 S BROAD ST,Doubletree Hotel,883702001,19107,1,1983,Hotel,565764.0,3774262,10622801.8,NaN,NaN,78.0,58.0,143.0,23640.1,3110.8,NaN
1,1160875.0,6195,700-50 CATHEDRAL RD,Cathedral East Apartments,881088500,19128,1,1964,Multifamily Housing,92156.0,5140660,4181930.1,NaN,NaN,84.0,58.0,87.2,2190.6,355.3,NaN
2,680100.7,6119,8020 ALGON AVE,8020 Algon,881183500,19152,1,1964,Multifamily Housing,50840.0,5029250,1824667.9,NaN,NaN,95.0,49.3,79.7,1021.8,174.9,NaN
3,1564263.5,4993,926-76 E CHELTEN AVE,Pastorius,785099400,19138,1,1967,K-12 School,75318.0,3337464,2648097.0,NaN,NaN,86.0,55.9,102.1,3146.3,320.0,NaN
4,584826.7,5327,6951 NEW STATE RD,MARJAM SUPPLY,884182000,19135,1,1945,Distribution Center,106400.0,3813146,467121.4,NaN,NaN,91.0,9.9,21.9,612.2,91.9,NaN


In [54]:
print(yr2014_subset_df.columns)

Index(['natural_gas_use_kbtu', 'portfolio_manager_id', 'postal_code',
       'property_name', 'source_eui_kbtu_ft', 'total_ghg_emissions_mtco2e',
       'district_steam_use_kbtu', 'site_eui_kbtu_ft',
       'primary_property_type_epa_calculated', 'number_of_buildings',
       'energy_star_score', 'location_1_address', 'year_built',
       'fuel_oil_2_use_kbtu', 'property_floor_area_buildings_and_parking_ft',
       'water_use_all_water_sources_kgal', 'electricity_use_kbtu', 'notes',
       'philadelphia_building_id'],
      dtype='object')


In [70]:
yr2014_subset_df.rename(columns={'natural_gas_use_kbtu': 'Natural Gas Use', 'portfolio_manager_id': 'Port Manager ID', 'postal_code': 'Postal Code', 'property_name': 'Property Name', 'source_eui_kbtu_ft': 'Source EUI', 'total_ghg_emissions_mtco2e': 'Total Greenhouse Gas Emission', 'district_steam_use_kbtu': 'Steam Use', 'site_eui_kbtu_ft': 'Site EUI', 'primary_property_type_epa_calculated': 'Property Type', 'number_of_buildings': 'Num of Buildings', 'energy_star_score': 'Energy Star Score', 'location_1_address': 'Street Address', 'year_built': 'Year Built', 'fuel_oil_2_use_kbtu': 'Fuel Use', 'property_floor_area_buildings_and_parking_ft': 'Square Footage', 'water_use_all_water_sources_kgal': 'Water Use', 'electricity_use_kbtu':'Electricity Use', 'notes': 'Notes', 'philadelphia_building_id': 'Building ID'}, inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [71]:
yr2014_subset_df.head()

,Natural Gas Use,Port Manager ID,Postal Code,Property Name,Source EUI,Total Greenhouse Gas Emission,Steam Use,Site EUI,Property Type,Num of Buildings,Energy Star Score,Street Address,Year Built,Fuel Use,Square Footage,Water Use,Electricity Use,Notes,Building ID
0,"5,108,484.20",3330960,19120.0,Lowell,245.2,651.0,Not Available,136.5,K-12 School,1,11,450 W NEDRO AVE,1914,Not Available,58200.0,1493.9,2836085.4,NaN,785548000
1,"6,566,999.70",3952012,19116.0,10814 NE AVENUE,35.6,715.1,Not Available,21.4,Other - Services,1,Not Available,10814 NE AVENUE,1979,Not Available,217560.0,245.5,2736515.4,NaN,884240500
2,"243,912.00",3989966,19124.0,Juniata,34.1,88.9,Not Available,13.6,Self-Storage Facility,1,Not Available,"4123 ""G"" St.",2005,Not Available,59850.0,19.7,567623.4,NaN,884114205
3,"5,401,817.60",3330939,19143.0,Dobson,146.5,373.0,Not Available,115.1,K-12 School,1,12,4667 UMBRIA,1930,Not Available,52500.0,379.5,643237.4,NaN,784463600
4,"78,090.40",3330927,19128.0,Levering,121.5,546.1,Not Available,97.6,K-12 School,1,54,6000 RIDGE AVE,1929,"5,955,528.10",69475.0,284.1,747159.8,NaN,784378000


In [55]:
print(yr2013_subset_df.columns)

Index(['the_geom', 'lng', 'year_built',
       'electricity_use_grid_purchase_and_generated_from_onsite_renewab',
       'energy_star_score', 'portfolio_manager_id',
       'water_use_all_water_sources_kgal', 'district_steam_use_kbtu',
       'site_eui_kbtu_ft2', 'philadelphia_building_id', 'property_name',
       'the_geom_webmercator', 'geom_address', 'fuel_oil_2_use_kbtu',
       'number_of_buildings', 'property_floor_area_buildings_and_parking_ft',
       'postal_code', 'natural_gas_use_kbtu', 'notes',
       'total_ghg_emissions_mtco2e', 'primary_property_type_epa_calculated',
       'lat', 'source_eui_kbtu_ft2'],
      dtype='object')


In [76]:
yr2013_subset_df.rename(columns={'the_geom': 'Geometry', 'lng': 'Longitude', 'year_built': 'Year Built', 'electricity_use_grid_purchase_and_generated_from_onsite_renewab': 'Renewable Use', 'energy_star_score': 'Energy Star Score', 'portfolio_manager_id': 'Port Manager ID', 'water_use_all_water_sources_kgal': 'Water Use', 'district_steam_use_kbtu': 'Steam Use', 'site_eui_kbtu_ft2': 'Site EUI', 'philadelphia_building_id': 'Building ID', 'property_name': 'Property Name', 'the_geom_webmercator': 'Geom Webmercator', 'geom_address': 'Geom Address', 'fuel_oil_2_use_kbtu': 'Fuel Use', 'number_of_buildings': 'Num of Buildings', 'property_floor_area_buildings_and_parking_ft': 'Square Footage', 'postal_code':'Postal Code', 'natural_gas_use_kbtu': 'Natural Gas Use', 'notes': 'notes', 'total_ghg_emissions_mtco2e':'Total Greenhouse Gas Emission', 'primary_property_type_epa_calculated':'Property Type', 'lat':'Latitude','source_eui_kbtu_ft2':'Source EUI'}, inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [77]:
yr2013_subset_df.head()

,Geometry,Longitude,Year Built,Renewable Use,Energy Star Score,Port Manager ID,Water Use,Steam Use,Site EUI,Building ID,...,Fuel Use,Num of Buildings,Square Footage,Postal Code,Natural Gas Use,notes,Total Greenhouse Gas Emission,Property Type,latitude,Source EUI
0,0101000020E61000002CBA8FB8D4CA52C0C82FAB660CFE...,-75.169233,1940,2146869.3,25,3330868,1604.5,Not Available,94.0,784321800,...,Not Available,1,109651.0,19121.0,8157192.2,NaN,704.8,K-12 School,39.984753,139.6
1,0101000020E610000080FC0401C6CB52C0A0BF7A90E9F9...,-75.183960,1917,3916010.4,73,3443035,2900.9,1299111.2,67.8,773717200,...,Not Available,1,79200.0,19104.0,155304.3,NaN,619.1,Office,39.952440,177.0
2,0101000020E6100000DC15D69AE1CA52C068520C2805F9...,-75.170020,1905,13312010.1,1,3512105,262.0,Not Available,225.2,772614900,...,0,1,59101.0,19146.0,Not Available,NaN,1685.8,Office,39.945470,707.3
3,0101000020E6100000B09CFBE0B9BF52C0F07C5E429C0E...,-74.995720,1950,192780.4,100,3892836,28.4,Not Available,3.0,884233835,...,Not Available,1,64638.0,19116.0,0,NaN,24.4,Office,40.114144,9.4
4,0101000020E610000054D00AC543CA52C09851DFDEA7F9...,-75.160386,1925,2756896.0,44,3759390,985.2,558808.5,69.5,883311000,...,Not Available,1,51676.0,19107.0,274307.8,NaN,413.2,Office,39.950436,186.1


## 4. Start the merging chain process

# ERROR- CANT FIGURE OUT HOW TO DO THIS --THIS IS WHERE NOTEBOOK 5 COMES IN I THINK

### Merging 2013 with 2014

#### Step 1. Make sure all columns are the same

In [51]:
yr2013_subset_df.head()

,the_geom,lng,year_built,electricity_use_grid_purchase_and_generated_from_onsite_renewab,energy_star_score,portfolio_manager_id,water_use_all_water_sources_kgal,district_steam_use_kbtu,site_eui_kbtu_ft2,philadelphia_building_id,...,fuel_oil_2_use_kbtu,number_of_buildings,property_floor_area_buildings_and_parking_ft,postal_code,natural_gas_use_kbtu,notes,total_ghg_emissions_mtco2e,primary_property_type_epa_calculated,lat,source_eui_kbtu_ft2
0,0101000020E61000002CBA8FB8D4CA52C0C82FAB660CFE...,-75.169233,1940,2146869.3,25,3330868,1604.5,Not Available,94.0,784321800,...,Not Available,1,109651.0,19121.0,8157192.2,NaN,704.8,K-12 School,39.984753,139.6
1,0101000020E610000080FC0401C6CB52C0A0BF7A90E9F9...,-75.183960,1917,3916010.4,73,3443035,2900.9,1299111.2,67.8,773717200,...,Not Available,1,79200.0,19104.0,155304.3,NaN,619.1,Office,39.952440,177.0
2,0101000020E6100000DC15D69AE1CA52C068520C2805F9...,-75.170020,1905,13312010.1,1,3512105,262.0,Not Available,225.2,772614900,...,0,1,59101.0,19146.0,Not Available,NaN,1685.8,Office,39.945470,707.3
3,0101000020E6100000B09CFBE0B9BF52C0F07C5E429C0E...,-74.995720,1950,192780.4,100,3892836,28.4,Not Available,3.0,884233835,...,Not Available,1,64638.0,19116.0,0,NaN,24.4,Office,40.114144,9.4
4,0101000020E610000054D00AC543CA52C09851DFDEA7F9...,-75.160386,1925,2756896.0,44,3759390,985.2,558808.5,69.5,883311000,...,Not Available,1,51676.0,19107.0,274307.8,NaN,413.2,Office,39.950436,186.1


### Merging 13/14 with 2015

### Merging 13/14/15 with 2016

### Merging 13/14/15/16 with 2017